In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('C:/Users/Adm/Desktop/GoIT/stack-overflow-developer-survey-2023/survey_results_public.csv')
schema = pd.read_csv('C:/Users/Adm/Desktop/GoIT/stack-overflow-developer-survey-2023/survey_results_schema.csv')

### Скільки респондентів пройшли опитування?

In [ ]:
df.ResponseId.nunique()

### Скільки респондентів відповіли на всі запитання?

In [ ]:
questions = set(schema.qname.unique()) & set(df.columns)
df.dropna(subset=questions).shape[0]

### Які значення мір центральної тенденції для досвіду (WorkExp) респондентів?

In [ ]:
print(f'Mean: {round(df.WorkExp.mean(), 1)}')
print(f'Mode: {round(df.WorkExp.mode()[0], 1)}')
print(f'Median: {round(df.WorkExp.median(), 1)}')

### Скільки респондентів працює віддалено?

In [ ]:
df[df.RemoteWork == 'Remote'].shape[0]

### Який відсоток респондентів програмує на Python?

In [ ]:
# Первый вариант
df['worked_with_python'] = df.LanguageHaveWorkedWith.apply(lambda x: 1 if 'python' in str(x).lower() else 0)
round(df.worked_with_python.sum() / df.ResponseId.nunique(), 2)

In [ ]:
# Второй вариант вариант
df['worked_with_python'] = df.LanguageHaveWorkedWith.str.contains('python', case=False, na=False)
round(df.worked_with_python.sum() / df.ResponseId.nunique(), 2)

### Скільки респондентів навчалося програмувати за допомогою онлайн курсів?

In [ ]:
# Первый вариант
df['learned_with_online_courses'] = df.LearnCode.apply(lambda x: 1 if 'online courses' in str(x).lower() else 0)
df.learned_with_online_courses.sum()

In [ ]:
# Второй вариант
df['learned_with_online_courses'] = df.LearnCode.str.contains('online courses', case=False, na=False)
df.learned_with_online_courses.sum()

### Серед респондентів що програмують на Python в групуванні по країнам, яка середня та медіанна сума компенсації (ConvertedCompYearly) в кожній країні?

In [ ]:
py_progers_by_country = df[df.worked_with_python == 1].dropna(subset='ConvertedCompYearly') \
                                                      .groupby('Country', as_index=False) \
                                                      .agg({'ConvertedCompYearly': ['mean', 'median']}).droplevel(1, axis=1)
py_progers_by_country.columns = ['country', 'mean_comp', 'median_comp']
py_progers_by_country = py_progers_by_country.astype({'mean_comp': 'int', 'median_comp': 'int'})
py_progers_by_country

### Які рівні освіти мають 5 респондентів з найбільшою компенсацією?

In [ ]:
df[['ResponseId', 'EdLevel', 'ConvertedCompYearly']].sort_values(by='ConvertedCompYearly', ascending=False).reset_index(drop=True).head(5)

### В кожній віковій категорії, який відсоток респондентів програмує на Python?

In [ ]:
py_progers_share_by_age = df.groupby('Age', as_index=False).agg({'ResponseId': 'count', 'worked_with_python': 'sum'})
py_progers_share_by_age['work_with_python_share'] = round(py_progers_share_by_age.worked_with_python / py_progers_share_by_age.ResponseId, 2)
py_progers_share_by_age

### Серед респондентів що знаходяться в 75 перцентилі за компенсацією середнього і працюють віддалено, які індустрії є найрозповсюдженішими?

In [ ]:
df[(df.ConvertedCompYearly > df.ConvertedCompYearly.quantile(0.75)) & (df.RemoteWork == 'Remote')].Industry.value_counts().reset_index()

In [ ]:
online_course_learners = df[df['LearnCode'].str.contains('Online', case=False, na=False)]

num_online_course_learners = online_course_learners['ResponseId'].nunique()

print(num_online_course_learners)

In [ ]:
df.WorkExp.mode()

In [ ]:
df